In [1]:
import os
import sys
import time
import glob
import numpy as np
import logging
import argparse
import tensorflow as tf
import numpy as np
import utils
from tqdm import tqdm
import shutil
from model_search import Network
from architect_graph import Architect

In [2]:
import utils
from genotypes import Genotype

In [3]:
args = {
    "momentum": 0.9,
    "weight_decay": 3e-4,
    "arch_learning_rate": 3e-1,
    "momentum": 0.9,
    "grad_clip": 5,
    "learning_rate": 0.025,
    "learning_rate_decay": 0.97,
    "learning_rate_min": 0.0001,
    "num_batches_per_epoch": 2000,
    
    "unrolled": True,
    "epochs": 10,
    "train_batch_size": 8,
    "eval_batch_size": 8,
    "save": "EXP",
    "init_channels": 3,
    "num_layers": 3,
    "num_classes": 6,
    "crop_size": [4, 4],
    "save_checkpoints_steps": 100,
    "model_dir": 'gs://isro-nas-trial/abc',
    
    "use_tpu": True,
    "use_host_call": True,
    "tpu": 'unet-darts',
    "zone": 'us-central1-f',
    "project": "isro-nas"
}

class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)

args = Struct(**args)

### input_fn

In [4]:
def make_inp_fn(filename, mode, batch_size):
    
    def _input_fn():
        image_dataset = tf.data.TFRecordDataset(filename)
        W, H = 16, 16

        # Create a dictionary describing the features.  
        image_feature_description = {
            'name': tf.FixedLenFeature([], tf.string),  
            'label_encoded': tf.FixedLenFeature([], tf.string),
            'encoded': tf.FixedLenFeature([], tf.string)
        }
        def _parse_image_function(example_proto):
            # Parse the input tf.Example proto using the dictionary above.
            feature= tf.parse_single_example(example_proto, image_feature_description)
            image= feature['encoded']
            label = feature['label_encoded']
            name = feature['name']

            image = tf.image.decode_png(image, channels=3)
            label = tf.image.decode_png(label, channels=3)

            image = tf.cast(image, tf.float32)
            image = tf.image.resize(image, (W, H))
            label = tf.cast(label, tf.float32)
            label = tf.image.resize(label, (W, H))

            return image, label

        dataset = image_dataset.map(_parse_image_function)
        
        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None # indefinitely
            dataset = dataset.shuffle(buffer_size = 10 * batch_size)
        else:
            num_epochs = 1 # end-of-input after this

        dataset = dataset.repeat(num_epochs).batch(batch_size,  drop_remainder=True)

        return dataset
    
    return _input_fn

In [5]:
def make_inp_fn2(filename, mode, batch_size):
    
    def _input_fn(params):
        W, H = args.crop_size[0], args.crop_size[1]
        NUM_IMAGES = 20
        if mode == tf.estimator.ModeKeys.TRAIN or mode == tf.estimator.ModeKeys.EVAL:
            x_train = np.random.randint(0, 256, (NUM_IMAGES // batch_size, batch_size, W, H, 3)).astype(np.float32)
            y_train = np.random.randint(0, args.num_classes, (NUM_IMAGES // batch_size, batch_size, W, H, 1)).astype(np.float32)
            x_valid = np.random.randint(0, 256, (NUM_IMAGES // batch_size, batch_size, W, H, 3)).astype(np.float32)
            y_valid = np.random.randint(0, args.num_classes, (NUM_IMAGES // batch_size, batch_size, W, H, 1)).astype(np.float32)
            
            ds = (x_train, x_valid), (y_train, y_valid)
            dataset = tf.data.Dataset.from_tensor_slices(ds)
            num_epochs = None # indefinitely
            dataset = dataset.shuffle(buffer_size = 10 * batch_size)
        else:
            x_train = np.random.randint(0, 256, (NUM_IMAGES, W, H, 3)).astype(np.float32)
            y_train = np.random.randint(0, 6, (NUM_IMAGES, W, H, 1)).astype(np.float32)
            
            ds = (x_train, y_train)
            dataset = tf.data.Dataset.from_tensor_slices(ds)
            num_epochs = 1 # end-of-input after this
#         w, h, c = dataset.shape
        dataset = dataset.repeat(num_epochs)
        return dataset
    
    return _input_fn

### Model function

In [6]:
class GeneSaver(tf.estimator.SessionRunHook):
    def __init__(self, genotype):
        self.genotype = genotype
    
    def begin(self):
        self.global_step = tf.train.get_or_create_global_step()
    
    def end(self, session):
        normal_gene_op = self.genotype.normal
        reduce_gene_op = self.genotype.reduce
        
        self.global_step = session.run(self.global_step)
        normal_gene = session.run(normal_gene_op)
        reduce_gene = session.run(reduce_gene_op)
        
        genotype = Genotype(
            normal=normal_gene, normal_concat=self.genotype.normal_concat,
            reduce=reduce_gene, reduce_concat=self.genotype.reduce_concat
        )
        
        filename = 'final_genotype.{}'.format((self.global_step))
        tf.logging.info("Saving Genotype for step: {}".format(str(self.global_step)))
        utils.write_genotype(genotype, filename)

In [13]:
def model_fn(features, labels, mode, params):
    criterion = tf.losses.softmax_cross_entropy
    model = Network(C=args.init_channels, net_layers=args.num_layers, criterion=criterion, num_classes=args.num_classes)
    global_step = tf.train.get_global_step()
    learning_rate_min = tf.constant(args.learning_rate_min)
    
    learning_rate = tf.train.exponential_decay(
        args.learning_rate,
        global_step,
        decay_rate=args.learning_rate_decay,
        decay_steps=args.num_batches_per_epoch,
        staircase=True,
    )
    
    lr = tf.maximum(learning_rate, learning_rate_min)
#     tf.summary.scalar('learning_rate', lr)
    
    optimizer = tf.train.MomentumOptimizer(lr, args.momentum)
    criterion = tf.losses.softmax_cross_entropy
    
    if(args.use_tpu):
        optimizer = tf.tpu.CrossShardOptimizer(optimizer)
        
    eval_hooks = None
    
    loss = None
    train_op = None
    eval_metric_ops = None
    prediction_dict = None
    export_outputs = None
    # 2. Loss function, training/eval ops
    if mode == tf.estimator.ModeKeys.TRAIN:
        (x_train, x_valid) = features
        (y_train, y_valid) = labels

        preds = model(x_train)
        architect = Architect(model, args)
        # architect step
        architect_step = architect.step(input_train=x_train,
                                        target_train=y_train,
                                        input_valid=x_valid,
                                        target_valid=y_valid,
                                        unrolled=args.unrolled,
                                        )


        w_var = model.get_thetas()
        loss = model._loss(preds, y_train)
        grads = tf.gradients(loss, w_var)
        clipped_gradients, norm = tf.clip_by_global_norm(grads, args.grad_clip)
        train_op = optimizer.apply_gradients(zip(clipped_gradients, w_var), global_step=tf.train.get_global_step())
        
        b, w, h, c = y_train.shape
        y = tf.reshape(tf.cast(y_train, tf.int64), (b, w, h))
        y = tf.one_hot(y, args.num_classes, on_value=1.0, off_value=0.0)
        
        metric_fn = lambda y, preds: {
            'miou': tf.metrics.mean_iou(
                    labels=y,
                    predictions=tf.round(preds),
                    num_classes=args.num_classes
                ),
        
            'acc': tf.metrics.accuracy(labels=y, 
                                      predictions=tf.round(preds))
        }
        eval_metric_ops = (metric_fn, [y, preds])
        
        if params['use_host_call']:
            def host_call_fn(global_step, learning_rate):
                # Outfeed supports int32 but global_step is expected to be int64.
                global_step = tf.reduce_mean(global_step)
                global_step = tf.cast(global_step, tf.int64)

                with (tf.contrib.summary.create_file_writer(params['model_dir']).as_default()):
                    with tf.contrib.summary.always_record_summaries():
                        tf.contrib.summary.scalar(
                            'learning_rate', tf.reduce_mean(learning_rate),
                            step=global_step)
                return tf.contrib.summary.all_summary_ops()
            
            global_step_t = tf.reshape(global_step, [1])
            learning_rate_t = tf.reshape(learning_rate, [1])
            host_call = (host_call_fn,
                           [global_step_t, learning_rate_t])

    elif mode == tf.estimator.ModeKeys.EVAL:
        genotype = model.genotype()
        gene_saver = GeneSaver(genotype)
        eval_hooks = [gene_saver]
        
        (x_train, x_valid) = features
        (y_train, y_valid) = labels
        preds = model(x_valid)
        loss = model._loss(preds, y_valid)
        
        b, w, h, c = y_valid.shape
        y = tf.reshape(tf.cast(y_valid, tf.int64), (b, w, h))
        y = tf.one_hot(y, args.num_classes, on_value=1.0, off_value=0.0)
        
        metric_fn = lambda y, preds: {
            'miou': tf.metrics.mean_iou(
                    labels=y,
                    predictions=tf.round(preds),
                    num_classes=args.num_classes
                ),
        
            'acc': tf.metrics.accuracy(labels=y, 
                                      predictions=tf.round(preds))
        }
        eval_metric_ops = (metric_fn, [y, preds])
        
    elif mode == tf.estimator.ModeKeys.PREDICT:
        x = features
        y = labels
        preds = model(x)
        prediction_dict = {"predictions": preds}
        export_outputs = {"predict_export_outputs": tf.estimator.export.PredictOutput(outputs = preds)}
    
    # 5. Return EstimatorSpec
    return tf.estimator.tpu.TPUEstimatorSpec(
        mode = mode,
        predictions = prediction_dict,
        loss = loss,
        train_op = train_op,
        eval_metrics = eval_metric_ops,
        export_outputs = export_outputs,
        evaluation_hooks=eval_hooks,
        host_call=host_call
    )

### Estimator

In [14]:
# Create functions to read in respective datasets
def get_train():
    return make_inp_fn2(filename = '../../datasets/infer/infer-00000-00007.tfrecords',
                        mode = tf.estimator.ModeKeys.TRAIN,
                        batch_size = args.train_batch_size)

In [15]:
# Create serving input function
def serving_input_fn():
    feature_placeholders = {
      IMAGE_LOC: tf.placeholder(tf.float32, [None])
    }
    
    feature_placeholders['IMAGES'] = tf.placeholder(tf.float32, [None, args.crop_size[0], args.crop_size[1], args.init_channels])
    
    features = {
    key: tf.expand_dims(tensor, -1)
    for key, tensor in feature_placeholders.items()
    }

    return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)


In [16]:
tpu_cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(
      tpu=args.tpu,
      zone=args.zone,
      project=args.project)

In [17]:
config = tf.estimator.tpu.RunConfig(
      cluster=tpu_cluster_resolver,
      model_dir=args.model_dir,
      save_checkpoints_steps=args.save_checkpoints_steps,
      tpu_config=tf.contrib.tpu.TPUConfig(
          iterations_per_loop=1000,
          num_shards=None))

In [18]:
estimator = tf.estimator.tpu.TPUEstimator(
    use_tpu=True,
    model_fn=model_fn,
    config=config,
    train_batch_size=args.train_batch_size,
    eval_batch_size=args.eval_batch_size,
    params=vars(args)
)
# tf.estimator.Estimator(model_fn = model_fn, 
#                      config=config)
estimator.train(input_fn = get_train(), steps = 1)

INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.240.1.18:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f586844f410>, '_model_dir': 'gs://isro-nas-trial/abc', '_protocol': None, '_save_checkpoints_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=None, num_cores_per_replica=None, per_host_input_for_training=2, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None, eval_training_input_configuration=2, experimental_host_call_every_n_steps=1), '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_session_creation_timeout_se

In [19]:
# Create custom estimator's train and evaluate function
def train_and_evaluate(output_dir, estimator):
    train_spec = tf.estimator.TrainSpec(input_fn = get_train(),
                                    max_steps = 1000)
    exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)
    eval_spec = tf.estimator.EvalSpec(input_fn = get_train(),
                                  steps = None, throttle_secs=600)
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

In [30]:
train_and_evaluate(args.model_dir, estimator)

NameError: name 'train_and_evaluate' is not defined

## Extra Test

In [14]:
criterion = tf.losses.softmax_cross_entropy
model = Network(C=args.init_channels, net_layers=args.num_layers, criterion=criterion, num_classes=args.num_classes)

In [15]:
(x_train, x_valid), (y_train, y_valid) = get_train()(vars(args)).make_one_shot_iterator().get_next()

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


In [16]:
logits = model(x_train)

In [14]:
loss = model._loss(logits, y_train)

(<tf.Tensor 'one_hot:0' shape=(8, 4, 4, 6) dtype=float32>, <tf.Tensor 'network/softmaxConv/softmax/Softmax:0' shape=(8, 4, 4, 6) dtype=float32>)


In [16]:
optimizer = tf.train.GradientDescentOptimizer(0.001)
optimizer.minimize(loss, var_list=model.get_thetas())

<tf.Operation 'GradientDescent' type=NoOp>

In [17]:
y_train

<tf.Tensor 'IteratorGetNext:2' shape=(8, 4, 4, 1) dtype=float32>

In [18]:
logits

<tf.Tensor 'network/softmaxConv/softmax/Softmax:0' shape=(8, 4, 4, 6) dtype=float32>

In [20]:
b, w, h, c = y_train.shape
y = tf.reshape(tf.cast(y_train, tf.int64), (b, w, h))
y = tf.one_hot(y, args.num_classes, on_value=1.0, off_value=0.0)

In [21]:
y

<tf.Tensor 'one_hot:0' shape=(8, 4, 4, 6) dtype=float32>

In [53]:
miou = tf.metrics.mean_iou(
    labels=y,
    predictions=tf.round(logits),
    num_classes=args.num_classes
)

In [50]:
b = tf.metrics.mean_iou(
    labels=tf.constant([[1, 0, 0], [0, 1, 0]]),
    predictions=tf.constant([[1, 0, 0], [0, 0.99999, 0]]),
    num_classes=2
)

In [54]:
local_op = tf.local_variables_initializer()
global_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run([local_op, global_op])
    out = sess.run(miou)